## Анализ базы данных сервиса чтения книг

**Цель** - формирование концепции нового продукта на основании анализа БД выкупленного сервиса.

В нашем распоряжении БД, содержащая следующие данные:

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# подключение к БД
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Знакомство с данными.

**Таблица `books`**

In [3]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()

# для удобства и экономии, в т.ч. времени создадим функцию, отображающую для нас содержимое таблиц/последующих запросов

def execute_query(query):
    display(pd.io.sql.read_sql(sql=text(query), con = con))
    
execute_query(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `authors`**

In [4]:
author = '''SELECT * FROM authors LIMIT 5'''
execute_query(author)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `publishers`**

In [5]:
publisher = '''SELECT * FROM publishers LIMIT 5'''
execute_query(publisher)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `ratings`**

In [6]:
rating = '''SELECT * FROM ratings LIMIT 5'''
execute_query(rating)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `reviews`**

In [7]:
review = '''SELECT * FROM reviews LIMIT 5'''
execute_query(review)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Задания.

#### 1. Посчитаем, сколько книг вышло после 1 января 2000 года.

In [8]:
books_to_january = '''SELECT COUNT(*) as count_books
FROM books
WHERE publication_date::date >= '2000-01-01'
'''

execute_query(books_to_january)

,count_books
0,821


**Вывод:** с наступлением 2000 г в каталог сервиса было добавлено - 821 книга/брошюра.

#### 2. Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [9]:
about_book = '''SELECT bk.book_id,
bk.title,
COUNT(DISTINCT review_id) as count_review,
AVG(rating) as avg_rating
FROM books bk
LEFT JOIN ratings rt ON rt.book_id = bk.book_id
LEFT JOIN reviews rw ON rw.book_id = bk.book_id
GROUP BY bk.book_id, bk.title
ORDER BY count_review DESC
'''

execute_query(about_book)

,book_id,title,count_review,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


**Вывод:** каталог включает 1000 произведений. Кол-во обзоров - от 0 до 7. Средняя оценка - от 1 до 5.  
Среднюю оценку каждой книги можно увидеть по результатам запроса. Видим, что кол-во обзоров не влияет на оценку,т.е. она может быть выше при наличии мЕньшего кол-ва обзоров. Оценка субъективна и индивидуальна. 

#### 3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [10]:
pulishers_without_brochures = '''SELECT publisher,
COUNT(*) as cnt_books
FROM publishers pb
LEFT JOIN books bk ON pb.publisher_id = bk.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY cnt_books DESC
LIMIT 1
'''
execute_query(pulishers_without_brochures)

,publisher,cnt_books
0,Penguin Books,42


**Вывод:** Наибольшую долю в каталоге(брошюры исключены) занимают книги издательства Penguin Books.

#### 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [11]:
autor_with_max_rating = '''
WITH n AS ( SELECT a.author, 
       bk.book_id,
       COUNT(*),
       AVG(rt.rating) AS avg_rating
FROM ratings rt
LEFT JOIN books bk ON rt.book_id = bk.book_id
LEFT JOIN authors a ON bk.author_id = a.author_id
GROUP BY a.author, bk.book_id
HAVING COUNT(*) >= 50
ORDER BY avg_rating DESC
)
SELECT author,
    AVG(avg_rating) AS rating
FROM n
GROUP BY author
ORDER BY rating DESC
LIMIT 1
'''
execute_query(autor_with_max_rating)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.283844


**Вывод:** автором с самой высокой средней оценкой книг (учли книги с 50 и более оценками) является J.K. Rowling/Mary GrandPré - автор и иллюстратор книг о Гарри Поттере. 

#### 5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
#  присоединение дало нам возможность добавить и тех,кто поставил 48 оценок, не написав при этом ни одного обзора
#  в данных таких не оказалось, поэтому после присоединения среднее не изменилось

avg_review = '''
SELECT
    AVG(cnt_rw) as avg_review
FROM (
    SELECT
        usernames.username,
        COUNT(*) as cnt_rw
    FROM (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(*) > 48
    ) AS usernames
    LEFT JOIN reviews rw ON rw.username = usernames.username
    GROUP BY usernames.username
) m
ORDER BY avg_review DESC
'''

execute_query(avg_review)


,avg_review
0,24.0


**Вывод:** В среднем, активные пользователи, поставившие более 48-ти оценок книгам/брошюрам, написали обзоры на 24 произведения.

### Общее заключение.

- с наступлением 2000 г в каталог сервиса было добавлено 821 книга/брошюра;
- каталог включает 1000 произведений; 
- кол-во обзоров - от 0 до 7; средняя оценка - от 1 до 5;
- кол-во обзоров не влияет на оценку,т.е. она может быть выше при наличии мЕньшего кол-ва обзоров;
- наибольшую долю в каталоге(брошюры исключены) занимают книги издательства Penguin Books;
- автором с самой высокой средней оценкой книг (учли книги с 50 и более оценками) является J.K. Rowling/Mary GrandPré - автор и иллюстратор книг о Гарри Поттере. 

Возможные рекомендации.

Оценка БД выделила наиболее популярные книги/авторов среди пользователей/читателей. Среди них, в т.ч. имеющие от 3-х и более частей. 
Для нового продукта можно для таких произведений добавить аудио-версии, для книг с несколькими частями 1-2 будут подарком за подписку, далее на платной основе. 

Аудио-версии, как и текстовые стоит иметь на нескольких языках + изучить вопрос популярности озвучки тем или иным актером/писателем и т.д. и добавлять именно их.

Также бонусами для первых пользователей может быть, например, месячная подписка на какой-либо сервис для просмотра киноверсий выбранных книг. Т.е. можно сделать маркетинговое предложение одному из таких сервисов.